In [2]:
import tarfile
import os

# Define the file path and the extraction path
file_path = 'ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz'
extract_path = 'model'

# Extract the tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

# List the extracted files
extracted_files = os.listdir(extract_path)
print(extracted_files)


['ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18']


In [3]:
extracted_dir_path = os.path.join(extract_path, 'ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18')
extracted_dir_contents = os.listdir(extracted_dir_path)
print(extracted_dir_contents)


['model.ckpt.meta', 'pipeline.config', 'frozen_inference_graph.pb', 'model.ckpt.index', 'model.ckpt.data-00000-of-00001', 'model.tflite']


In [11]:
import tensorflow as tf

# Load the frozen inference graph
model_path = os.path.join(extracted_dir_path,'frozen_inference_graph.pb')

# Load the model
def load_frozen_model(model_file):
    with tf.io.gfile.GFile(model_file, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

# Load the frozen model
graph_def = load_frozen_model(model_path)

# Define layer types to filter
layer_types_to_print = [
    'Conv2D', 'DepthwiseConv2dNative', 'FusedBatchNorm', 
    'Relu', 'MatMul', 'Add', 'Identity', 'MaxPool', 'AvgPool'
]  # Add or modify layer types as needed

# Function to filter and print specific layers
def print_specific_layers(graph_def, layer_types):
    filtered_layers = []
    for node in graph_def.node:
        if any(layer_type in node.op for layer_type in layer_types):
            filtered_layers.append(f"Layer Name: {node.name}, Layer Type: {node.op}")
    return filtered_layers

# Get the filtered layers
filtered_layers = print_specific_layers(graph_def, layer_types_to_print)

# Print number of filtered layers
print(f"Total number of filtered layers: {len(filtered_layers)}")

# Print filtered layers (optional)
for layer in filtered_layers:
    print(layer)


Total number of filtered layers: 1209
Layer Name: FeatureExtractor/MobilenetV2/MobilenetV2/input, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/Conv/weights/read, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/Conv/Conv2D, Layer Type: Conv2D
Layer Name: FeatureExtractor/MobilenetV2/Conv/BatchNorm/beta/read, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/Conv/BatchNorm/gamma/read, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/Conv/BatchNorm/moving_mean/read, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/Conv/BatchNorm/moving_variance/read, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/Conv/BatchNorm/FusedBatchNormV3, Layer Type: FusedBatchNormV3
Layer Name: FeatureExtractor/MobilenetV2/Conv/Relu6, Layer Type: Relu6
Layer Name: FeatureExtractor/MobilenetV2/expanded_conv/input, Layer Type: Identity
Layer Name: FeatureExtractor/MobilenetV2/expanded_conv/depthwise/depthwise_weights/read, Layer Type